In [1]:
%matplotlib inline

# Welcome to Exercise 07
1) OCR 오타 -> 원본 seq2seq 짜기<br>
2) inference 코드 짜기

# 1) OCR 오타 -> 원본 seq2seq 

## Data Generation

1) `PIL`을 통하여 텍스트가 있는 그림을 그리고 jpg로 저장<br>
2) `easyocr` 패키지를 사용하여 저장한 jpg를 ocr로 읽기<br>
3) **input** : ocr 결과, **output** : 원래 텍스트 로 데이터셋 만들기

In [2]:
# !pip install easyocr

In [3]:
from PIL import Image, ImageDraw, ImageFont
from glob import glob
import easyocr

In [4]:
corpus = [open(_).readline() for _ in glob('data/scraping/economic/*')]
corpus = ' '.join(corpus)
corpus = corpus.split()

In [5]:
reader = easyocr.Reader(['ko']) # need to run only once to load model into memory

In [6]:
word_list = []
for word in corpus[:1000]:
    if word:
        font = ImageFont.truetype('data/font/handwriting.TTF', 20)
        image = Image.new('RGB', (300,300), color='White')
        draw = ImageDraw.Draw(image)
        draw.text((50, 50), word, 50, font=font)
        image.save('data/ocr/image.jpg')
        result = reader.readtext('data/ocr/image.jpg')
        if result:
            ocr_result = result[0][1]
            if word != ocr_result:
                word_list.append((ocr_result그리고input word))

In [7]:
word_list[:2], len(word_list)

([('국나', '국내'), ('이긴희', '이건희')], 312)

## Data Preprocess
`torchtext` 없이는 이와 같은 일련의 작업을 해주어야 했다.<br>
1) 자모 분리하기 (word -> 자모로 토큰화)<br>
2) token 숫자화 <br>
3) SRC field는 `[sos]`로 시작해서 `[eos]`까지, <br>
4) TRG field는 `[sos]`로 시작해서 `[eos]` 토큰으로 전처리하기<br>
4) batch로 묶고 max_len으로 패딩하기 <br>

그런데 `torchtext`를 사용하면, `Field`를 정의하고 이를 `TabularDataset`에 넣어줌으로서, 이와 같은 전처리를 몇 줄의 코드로 구현할 수 있다<br>


In [8]:
import sys

자모 분리 util을 위해서 `hangul-utils`(https://github.com/kaniblu/hangul-utils) 를 다운 받아주고 경로에 풀어줍시다<br>
Mecab 형태소 분석기를 쓰지 않을 것이기 때문에 `Readme.md`에 있는 JPype1 설치 등은 신경쓰지 않아도 됩니다

In [9]:
sys.path.append('/home/long8v/hangul-utils/')

In [10]:
from hangul_utils import split_syllable_char, split_syllables, join_jamos

In [11]:
split_syllables('안녕')

'ㅇㅏㄴㄴㅕㅇ'

In [12]:
# hangul_utils는 한글만 자모 분리하고 숫자 등은 에러를 내기 때문에 위와 같이 한번 더 감싸줌
def split_syllabes_wo_error(chars):
    splitted = ''
    for char in chars:
        try:
            splitted += split_syllables(char)
        except:
            splitted += char
    return splitted

In [13]:
from torchtext.data import Field, BucketIterator, TabularDataset
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [14]:
import pandas as pd 

In [15]:
train_data, test_data = train_test_split(word_list)
train_data, valid_data = train_test_split(train_data)

### csv saving

In [16]:
pd.DataFrame(train_data, columns=['src', 'trg']).to_csv('data/ocr_train.csv', index=False)
pd.DataFrame(valid_data, columns=['src', 'trg']).to_csv('data/ocr_valid.csv', index=False)
pd.DataFrame(test_data, columns=['src', 'trg']).to_csv('data/ocr_test.csv', index=False)

###  define field

In [17]:
SRC = Field(tokenize = lambda e: split_syllabes_wo_error(e), # 캐릭터 단위
            tokenizer_language="kor",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

TRG = Field(tokenize = lambda e: split_syllabes_wo_error(e), # 캐릭터 단위
            tokenizer_language="kor",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

/home/long8v/anaconda3/envs/long36v/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


### TabularDataset loading

In [18]:
def TabularDatset_w_csv(path):
    return TabularDataset(path, format='csv', skip_header=True, fields=[('src', SRC), ('trg', TRG)])

In [19]:
train_dataset = TabularDatset_w_csv('data/ocr_train.csv')
valid_dataset = TabularDatset_w_csv('data/ocr_valid.csv')
test_dataset = TabularDatset_w_csv('data/ocr_test.csv')

/home/long8v/anaconda3/envs/long36v/lib/python3.6/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/home/long8v/anaconda3/envs/long36v/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


이제 ``train_data`` 를 정의했으니, ``torchtext`` 의 ``Field`` 에 있는 엄청나게 유용한 기능을 보게 될 것입니다.<br> 바로 ``build_vocab`` 메소드(method)로 각 언어와 연관된 어휘들을 만들어 낼 것입니다.

In [20]:
SRC.build_vocab(train_dataset)
TRG.build_vocab(train_dataset)

###  ``BucketIterator``

마지막으로 사용해 볼 ``torchtext`` 에 특화된 기능은 바로 ``BucketIterator`` 입니다.
첫 번째 인자로 ``TranslationDataset`` 을 전달받기 때문에 사용하기가 쉽습니다. 문서에서도 볼 수 있듯
이 기능은 <b>비슷한 길이의 예제들을 묶어주는 반복자(iterator)</b>를 정의합니다. 각각의 새로운 에포크(epoch)마다
새로 섞인 결과를 만드는데 필요한 패딩의 수를 최소화 합니다. 버케팅 과정에서 사용되는 저장 공간을 한번 살펴보시기 바랍니다.

In [21]:
from torchtext.data import Iterator, BucketIterator

In [22]:
BATCH_SIZE = 16
device = 'cuda'
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size = BATCH_SIZE,
    device = device,
    sort_key=lambda e: len(e.src))

/home/long8v/anaconda3/envs/long36v/lib/python3.6/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [23]:
for _ in train_iterator:
    print('-'*50)
    print(_.trg)
    print('-'*50)
    break

--------------------------------------------------
tensor([[ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
        [14,  4,  4, 16, 11, 33, 71,  7,  7,  7, 11,  8,  4, 46, 11, 35],
        [ 6, 13, 18, 12,  6, 24, 12, 10, 12,  9, 22,  6, 13, 18, 22, 24],
        [ 7,  7, 15, 11,  5, 37, 15, 19, 23, 42,  5,  8,  8,  4,  5, 26],
        [ 6,  9, 15, 27, 20, 16, 13,  4, 17,  6,  7,  7, 10, 11,  7, 29],
        [ 4,  4,  6,  7,  6, 12,  5, 17, 11,  5,  9, 18,  5,  9,  9,  3],
        [30, 48, 44,  6, 24,  5,  9, 14,  6,  8,  3,  4, 11,  3, 15,  1],
        [ 3,  4, 20,  4,  3,  4, 15, 10, 15, 40,  1,  3, 41,  1, 18,  1],
        [ 1,  3,  6,  4,  1, 41, 13, 20,  3,  3,  1,  1,  3,  1, 14,  1],
        [ 1,  1, 24, 13,  1, 28,  8, 18,  1,  1,  1,  1,  1,  1, 10,  1],
        [ 1,  1,  3,  3,  1, 31, 18,  3,  1,  1,  1,  1,  1,  1,  5,  1],
        [ 1,  1,  1,  1,  1,  8, 30,  1,  1,  1,  1,  1,  1,  1, 21,  1],
        [ 1,  1,  1,  1,  1,  6,  3,  1,  1,  1,  1,  1,  1, 

/home/long8v/anaconda3/envs/long36v/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


이 반복자들은 ``DataLoader`` 와 마찬가지로 호출할 수 있습니다. 아래 ``train`` 과 
``evaluation`` 함수에서 보면, 다음과 같이 간단히 호출할 수 있음을 알 수 있습니다 <br>
각 ``batch`` 는 ``src`` 와 ``trg`` 속성을 가지게 됩니다.
```
   src = batch.src
   trg = batch.trg
```

### ``nn.Module`` 과 ``Optimizer`` 정의하기

대부분은 ``torchtext`` 가 알아서 해줍니다 : 데이터셋이 만들어지고 반복자가 정의되면, 이 튜토리얼에서
우리가 해야 할 일이라고는 그저 ``nn.Module`` 와 ``Optimizer`` 를 모델로서 정의하고 훈련시키는 것이 전부입니다.


이 튜토리얼에서 사용할 모델은 *Neural Machine Translation by Jointly Learning to Align and Translate*에서 설명하고 있는 구조를 따르고 있습니다.

참고 : 이 튜토리얼에서 사용하는 모델은 언어 번역을 위해 사용할 예시 모델입니다. 이 모델을 사용하는 것은
이 작업에 적당한 표준 모델이기 때문이지, 번역에 적합한 모델이기 때문은 아닙니다. 여러분이 최신 기술 트렌드를
잘 따라가고 있다면 잘 아시겠지만, 현재 번역에서 가장 뛰어난 모델은 Transformers입니다.  이 튜토리얼의 모델이 사용하는 "attention" 은 Transformer 모델에서 제안하는
멀티 헤드 셀프 어텐션(multi-headed self-attention) 과는 다르다는 점을 알려드립니다.



In [24]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

In [25]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden

In [26]:
class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)

In [27]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dTabularDataset 정의im
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)
        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted_encoder_rep = torch.bmm(a, encoder_outputs)
        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)
        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))
        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)
        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)
        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)
        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)

In [28]:
import torch

In [29]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # 디코더로의 첫 번째 입력은 <sos> 토큰입니다.
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs

In [30]:
device = 'cuda'

In [31]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)
model = Seq2Seq(enc, dec, device).to(device)

In [32]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

In [33]:
model.apply(init_weights)

optimizer = optim.Adam(model.parameters())

In [34]:
def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 111,858 trainable parameters


참고 : 언어 번역의 성능 점수를 기록하려면, ``nn.CrossEntropyLoss`` 함수가 단순한
패딩을 추가하는 부분을 무시할 수 있도록 해당 색인들을 알려줘야 합니다.



In [35]:
PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

마지막으로 이 모델을 훈련하고 평가합니다 :



In [36]:
import math
import time

In [37]:
def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()
        output = model(src, trg)
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [38]:
def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():
        for _, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            output = model(src, trg, 0) #turn off teacher forcing
            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [39]:
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [40]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Epoch: 01 | Time: 0m 0s
	Train Loss: 4.286 | Train PPL:  72.643
	 Val. Loss: 4.238 |  Val. PPL:  69.290
Epoch: 02 | Time: 0m 0s
	Train Loss: 4.024 | Train PPL:  55.900
	 Val. Loss: 3.688 |  Val. PPL:  39.977
Epoch: 03 | Time: 0m 0s
	Train Loss: 3.564 | Train PPL:  35.303
	 Val. Loss: 3.595 |  Val. PPL:  36.398
Epoch: 04 | Time: 0m 0s
	Train Loss: 3.474 | Train PPL:  32.269
	 Val. Loss: 3.536 |  Val. PPL:  34.327
Epoch: 05 | Time: 0m 0s
	Train Loss: 3.453 | Train PPL:  31.589
	 Val. Loss: 3.535 |  Val. PPL:  34.308
Epoch: 06 | Time: 0m 0s
	Train Loss: 3.442 | Train PPL:  31.243
	 Val. Loss: 3.532 |  Val. PPL:  34.178
Epoch: 07 | Time: 0m 0s
	Train Loss: 3.440 | Train PPL:  31.186
	 Val. Loss: 3.525 |  Val. PPL:  33.943
Epoch: 08 | Time: 0m 0s
	Train Loss: 3.410 | Train PPL:  30.275
	 Val. Loss: 3.513 |  Val. PPL:  33.565
Epoch: 09 | Time: 0m 0s
	Train Loss: 3.406 | Train PPL:  30.130
	 Val. Loss: 3.506 |  Val. PPL:  33.312
Epoch: 10 | Time: 0m 0s
	Train Loss: 3.389 | Train PPL:  29.642


# 2) inference 코드

In [89]:
inference_data = [['브회장', ' '], ['삼성지바구조', ' '], ['저일모직', ' ']]

In [90]:
inference_data

[['브회장', ' '], ['삼성지바구조', ' '], ['저일모직', ' ']]

In [91]:
pd.DataFrame(inference_data).to_csv('data/ocr_inference.csv', index=False)

In [92]:
inference_ds = TabularDatset_w_csv('data/ocr_inference.csv')

In [433]:
max_index_list = []
for _ in inference_iterator:
    output, hidden = enc.forward(_.src.to(device))
    decoder_input = _.trg[0,:].to(device)
    decoder_hidden = hidden
    decoder_output = output
    max_len = 0

    while 1:
        if decoder_input == 1 or (max_len > 10):
            break
        decoder_output, decoder_hidden = dec.forward(decoder_input, decoder_hidden, output)  
        max_value, max_index = torch.topk(decoder_output, k=1, dim=-1)
        max_len += 1
        max_index_list.append(max_index)

In [432]:
for _ in inference_iterator:
    output = model.forward(_.src.to(device), _.trg.to(device))
    max_value, max_index = torch.topk(output, dim=1, k=1)
    print(max_index)

tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]]], device='cuda:0')
tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0,